In [1]:
# scripts/validators.py
import re

def validar_ruc_ec(ruc: str) -> bool:
    """
    Validación básica de RUC (Ecuador).
    - Debe ser numérico y tener 13 dígitos.
    - Los dos primeros dígitos: provincia (01-24)
    - Tercer dígito: < 6 para cédula natural; 6 para sociedades; 9 para sociedades públicas (reglas varían).
    - Validación de dígito verificador (algoritmo típico para cédula/RUC)
    Retorna True si pasa validaciones sintácticas y checksum; False en caso contrario.
    """
    ruc = re.sub(r'\D', '', ruc or "")
    if len(ruc) != 13:
        return False

    # provincia
    provincia = int(ruc[:2])
    if provincia < 1 or provincia > 24:
        return False

    tercer = int(ruc[2])
    if tercer >= 6 and tercer != 6 and tercer != 9:
        # para simplificar: si es > 6 y distinto de 9, se considera inválido en muchas reglas
        return False

    # Validar como cédula (primeros 10 dígitos) si tercero < 6
    def validar_cedula(cedula10):
        coef = [2,1,2,1,2,1,2,1,2]
        total = 0
        for i, c in enumerate(cedula10[:9]):
            val = int(c) * coef[i]
            if val >= 10:
                val = val - 9
            total += val
        ver = (10 - (total % 10)) % 10
        return ver == int(cedula10[9])

    # Si tercer <6 lo validamos como cédula (primeros 10 dígitos)
    if tercer < 6:
        return validar_cedula(ruc[:10])

    # Si tercer == 6 (sociedades), se puede aplicar otro checksum (simplificado)
    # Muchos implementan validación para sociedad: usar módulo 11 con coeficientes distintos.
    if tercer == 6:
        coef = [3,2,7,6,5,4,3,2,0]  # ejemplo; implementaciones varían según fuente
        suma = 0
        for i in range(9):
            suma += int(ruc[i]) * coef[i]
        resto = suma % 11
        ver = 11 - resto if resto != 0 else 0
        return ver == int(ruc[9])

    # Si tercer == 9 (entidades públicas), validación distinta:
    if tercer == 9:
        coef = [4,3,2,7,6,5,4,3,2]
        suma = 0
        for i in range(9):
            suma += int(ruc[i]) * coef[i]
        resto = suma % 11
        ver = 11 - resto if resto != 0 else 0
        return ver == int(ruc[9])

    return False
